# Exploration des données EcoTrajet

Analyse exploratoire complète des données disponibles via l'API Supabase.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../')

from src.utils.data_loader import data_loader

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Vue d'ensemble des données

In [ ]:
# Charger toutes les sources de données
stations_df = await data_loader.load_velib_stations()
history_df = await data_loader.load_velib_availability_history(days_back=7)
transport_df = await data_loader.load_transport_modes()
trips_df = await data_loader.load_user_trips(limit=1000)

print("=== RÉSUMÉ DES DONNÉES ===")
print(f"Stations Vélib': {len(stations_df)}")
print(f"Historique (7j): {len(history_df)} enregistrements")
print(f"Modes transport: {len(transport_df)}")
print(f"Trajets utilisateurs: {len(trips_df)}")

## 2. Analyse géographique des stations

In [ ]:
# Distribution géographique
plt.figure(figsize=(12, 8))
plt.scatter(stations_df['coordonnees_geo_lon'], stations_df['coordonnees_geo_lat'], 
           c=stations_df['capacity'], cmap='viridis', alpha=0.6)
plt.colorbar(label='Capacité')
plt.title('Distribution géographique des stations Vélib')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

## 3. Patterns temporels

In [ ]:
# Analyse temporelle complète
if not history_df.empty:
    history_df['hour'] = history_df['timestamp'].dt.hour
    history_df['day_of_week'] = history_df['timestamp'].dt.day_name()
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Par heure
    hourly_avg = history_df.groupby('hour')['numbikesavailable'].mean()
    axes[0,0].plot(hourly_avg.index, hourly_avg.values, marker='o')
    axes[0,0].set_title('Disponibilité moyenne par heure')
    axes[0,0].set_xlabel('Heure')
    axes[0,0].grid(True)
    
    # Par jour de la semaine
    daily_avg = history_df.groupby('day_of_week')['numbikesavailable'].mean()
    axes[0,1].bar(daily_avg.index, daily_avg.values)
    axes[0,1].set_title('Disponibilité moyenne par jour')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Distribution occupation
    history_df['occupation_rate'] = history_df['numbikesavailable'] / (history_df['numbikesavailable'] + history_df['numdocksavailable'])
    axes[1,0].hist(history_df['occupation_rate'].dropna(), bins=50, alpha=0.7)
    axes[1,0].set_title('Distribution du taux d\'occupation')
    axes[1,0].set_xlabel('Taux d\'occupation')
    
    # Types de vélos
    bike_types = ['mechanical', 'ebike']
    bike_counts = [history_df['mechanical'].sum(), history_df['ebike'].sum()]
    axes[1,1].pie(bike_counts, labels=bike_types, autopct='%1.1f%%')
    axes[1,1].set_title('Répartition types de vélos')
    
    plt.tight_layout()
    plt.show()

## 4. Qualité des données

In [ ]:
print("=== QUALITÉ DES DONNÉES ===")
print("\nStations Vélib':")
print(stations_df.isnull().sum())
print(f"Doublons: {stations_df.duplicated().sum()}")

if not history_df.empty:
    print("\nHistorique disponibilité:")
    print(history_df.isnull().sum())
    print(f"Doublons: {history_df.duplicated().sum()}")

if not trips_df.empty:
    print("\nTrajets utilisateurs:")
    print(trips_df.isnull().sum())
    print(f"Doublons: {trips_df.duplicated().sum()}")